FINANCIAL FAQ

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!ls


sample_data


In [ ]:
import pandas as pd
df = pd.read_csv('Financial-QA-10k.csv')
df.head()

,question,answer,context,ticker,filing
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA,2023_10K
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA,2023_10K
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA,2023_10K
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA,2023_10K
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA,2023_10K


In [ ]:
df.isnull().sum()

,0
question,2
answer,2
context,1
ticker,0
filing,0


In [ ]:
df.duplicated().sum()

np.int64(7)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  6998 non-null   object
 1   answer    6998 non-null   object
 2   context   6999 non-null   object
 3   ticker    7000 non-null   object
 4   filing    7000 non-null   object
dtypes: object(5)
memory usage: 273.6+ KB


In [ ]:
df = df.dropna(subset=['question', 'answer'])
df = df.drop_duplicates()

In [ ]:
df[['question','answer']].sample(5)

,question,answer
5037,By how much did Ford Pro's revenue increase fr...,$6.29 billion
1661,Where can one find the consolidated financial ...,The consolidated financial statements can be f...
2418,Where can detailed information about the commi...,"Under Note 13, included in Part II, Item 8"
5083,Where do the Financial Statements and Suppleme...,The Financial Statements and Supplementary Dat...
3299,When did AT&T acquire BellSouth Corporation?,AT&T acquired BellSouth Corporation in 2006.


In [ ]:
df['ticker'].value_counts()


,count
ticker,
JNJ,199
AAPL,100
TSLA,100
LULU,100
NVDA,100
...,...
HD,99
CB,99
GOOGL,98


In [ ]:
df['filing'].value_counts()

,count
filing,
2023_10K,6990


In [ ]:
df.drop(columns=['filing'], inplace=True)

In [ ]:
for col in ['question', 'answer', 'context']:
    df[col] = df[col].str.strip()

In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.head()

,question,answer,context,ticker
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA


In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv('faq_cleaned.csv')
df.head()

,question,answer,context,ticker
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...,NVDA
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...,NVDA
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget...",NVDA
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...,NVDA


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
question_embeddings = model.encode(df['question'].tolist(),show_progress_bar=True)

In [ ]:
import numpy as np
np.save('question_embeddings.npy', question_embeddings)

In [5]:
import numpy as np
#testing
user_question = "what area did nvidia initially focus on?"
user_question = user_question.strip()
user_question_embedding = model.encode([user_question])[0]

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarities = np.dot(question_embeddings, user_question_embedding) / (np.linalg.norm(question_embeddings, axis=1) * np.linalg.norm(user_question_embedding))

top_index = np.argmax(similarities)
top_faq = df.iloc[top_index]
print("best match:",top_faq['question'])
print("answer:",top_faq['answer'])
print("score:",similarities[top_index])

best match: What area did NVIDIA initially focus on before expanding to other computationally intensive fields?
answer: NVIDIA initially focused on PC graphics.
score: 0.8610607


DAY 3

In [1]:
import pandas as pd
df = pd.read_csv('faq_cleaned.csv')

In [6]:
import numpy as np

def match_faq(query, model, faq_embeddings, df, top_k=3):
    # preprocess query
    q = query.strip()

    # embed query
    q_emb = model.encode([q])[0]

    # cosine similarity with all FAQ questions
    sims = np.dot(faq_embeddings, q_emb) / (
        np.linalg.norm(faq_embeddings, axis=1) * np.linalg.norm(q_emb)
    )

    # get top-k indices
    top_idx = sims.argsort()[::-1][:top_k]

    # slice dataframe and scores
    top_faqs = df.iloc[top_idx][['question', 'answer']]
    top_scores = sims[top_idx]

    return top_faqs, top_scores

In [7]:
user_question = "What area did NVIDIA initially focus on?"
top_faqs, top_scores = match_faq(user_question, model, question_embeddings, df, top_k=3)

print("User question:", user_question)
for i, ((idx, row), score) in enumerate(zip(top_faqs.iterrows(), top_scores)):
    print(f"\nTop {i+1} match (score={score:.3f}):")
    print("Q:", row['question'])
    print("A:", row['answer'])

User question: What area did NVIDIA initially focus on?

Top 1 match (score=0.861):
Q: What area did NVIDIA initially focus on before expanding to other computationally intensive fields?
A: NVIDIA initially focused on PC graphics.

Top 2 match (score=0.743):
Q: What significant invention did NVIDIA create in 1999?
A: NVIDIA invented the GPU in 1999.

Top 3 match (score=0.669):
Q: What manufacturing strategy does NVIDIA employ for its products?
A: NVIDIA employs a fabless manufacturing strategy, relying on key suppliers for all phases of the manufacturing process including wafer fabrication, assembly, testing, and packaging.


In [10]:
# more evaluation loops
tests = [
    ("Did NVIDIA invent anything in 1999?",  456),
    ("What area did NVIDIA initially focus on?", 679),
]

correct_top1 = 0
correct_top3 = 0

for q, true_idx in tests:
    faqs, scores = match_faq(q, model, question_embeddings, df, top_k=3)
    predicted_indices = faqs.index.tolist()

    if predicted_indices[0] == true_idx:
        correct_top1 += 1
    if true_idx in predicted_indices:
        correct_top3 += 1

n = len(tests)
print("Top‑1 accuracy:", correct_top1 / n)
print("Top‑3 recall:", correct_top3 / n)

Top‑1 accuracy: 0.0
Top‑3 recall: 0.0


In [12]:
df[df['question'].str.contains("significant invention", case=False)]
df[df['question'].str.contains("initially focus on", case=False)]

,question,answer,context,ticker
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha...",NVDA


In [26]:
df[df['question'].str.contains("amd", case=False)]

,question,answer,context,ticker
2899,What factors might cause AMD's actual results ...,Factors that could cause AMD's actual results ...,"The forwardquctions, AMD's cash flows from ope...",AMD
2900,What are the five strategic pillars of AMD's b...,The five strategic pillars of AMD's business s...,AMD's strategy is focused on five strategic pi...,AMD
2901,What types of products does AMD offer in its d...,AMD's data center computing solutions portfoli...,AMD offers a deep portfolio of data center com...,AMD
2902,In what markets is AMD a leader in adaptive an...,AMD is a leader in adaptive and embedded compu...,AMD is a leader in adaptive and embedded compu...,AMD
2903,What performance features do AMD’s GPUs provid...,"AMD’s GPUs, including discrete GPUs and semi-c...","Our GPUs, including discrete GPUs, semi-custom...",AMD
2904,What types of CPUs does AMD currently offer fo...,AMD currently offers the AMD EPYC Series proce...,Our CPUs for server platforms currently includ...,AMD
2905,What advancements characterize the AMD EPYC 90...,The AMD EPYC 9004 Series processors are built ...,Our 4th Gen AMD EPYC 9004 Series processors ar...,AMD
2906,How does the AMD Instinct family of GPU accele...,"The AMD Instinct family of GPU accelerators, i...",Our AMD Instinct™ family of GPU accelerator pr...,AMD
2907,What are the different segments that AMD repor...,AMD has four reportable segments: the Data Cen...,Our four reportable segments are: •the Data Ce...,AMD
2908,What improvements do AMD’s Ryzen 7000 Series d...,The AMD Ryzen 7000 Series desktop processors a...,Our Ryzen 7000 Series desktop processors are b...,AMD


In [33]:
tests = [
    ("Did NVIDIA invent anything in 1999?",  2),
    ("What is Apple's focus category", 113),
    ("What is Kroger's feul sales", 6832),
    ("impact of COVID-19 resulted in what?", 58),
    ("How much did AMD spend on research?", 2964),
]

correct_top1 = 0
correct_top5 = 0

for q, true_idx in tests:
    faqs, scores = match_faq(q, model, question_embeddings, df, top_k=5)
    predicted_indices = faqs.index.tolist()

    if predicted_indices[0] == true_idx:
        correct_top1 += 1
    if true_idx in predicted_indices:
        correct_top5 += 1

n = len(tests)
print("Top‑1 accuracy:", correct_top1 / n)
print("Top‑5 recall:", correct_top5 / n)


Top‑1 accuracy: 0.6
Top‑5 recall: 0.8


In [35]:
for i, (q, true_idx) in enumerate(tests):
    faqs, scores = match_faq(q, model, question_embeddings, df, top_k=3)
    predicted_indices = faqs.index.tolist()

    print(f"\nTest {i+1}: '{q}'")
    print(f"Expected index: {true_idx}")
    print(f"Top indices: {predicted_indices}")
    print(f"Top question(s): {faqs['question'].tolist()}")

    if predicted_indices[0] == true_idx:
        print("✔️ Top‑1 match correct")
        correct_top1 += 1
    else:
        print("❌ Top‑1 match incorrect")

    if true_idx in predicted_indices:
        correct_top3 += 1



Test 1: 'Did NVIDIA invent anything in 1999?'
Expected index: 2
Top indices: [2, 53, 0]
Top question(s): ['What significant invention did NVIDIA create in 1999?', 'What year was NVIDIA reincorporated in Delaware?', 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?']
✔️ Top‑1 match correct

Test 2: 'What is Apple's focus category'
Expected index: 113
Top indices: [113, 119, 126]
Top question(s): ['In which product categories is Apple focused on expanding its market opportunities?', 'How does Apple expand the range of its offerings?', "What is Apple's vision regarding inclusion and diversity within its workforce?"]
✔️ Top‑1 match correct

Test 3: 'What is Kroger's feul sales'
Expected index: 6832
Top indices: [6800, 6797, 6795]
Top question(s): ['How does Kroger present its financial performance in its reporting?', 'What is the role of Kroger’s retail media business?', "What is the primary focus of Kroger’s 'Our Brands' products?"]
❌ To